Recreating the version of Figure 4.26 in the hard copy textbook.
This differs from the PDF found online and using for images.
Neither mention an edition number, publication year is 2010,
and ISBN is the same (978-0-19-856-576-5).
Online PDF is presumably wrong,
because Figure 4.26 (b) is the same as 4.29 (b).

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
model_config = {
    "end_time": 25550.,
    "total_population": 1e5,
    "infectious_seed": 1.,
}

parameters = {
    "latent_period": 8.,
    "infectious_period": 7.,
    "r0": 13.,
    "life_expectancy": 70.,
}

In [ ]:
def build_demog_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    model = CompartmentalModel(
        times=(0, config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["total_population"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"],
        }
    )
    
    # Transitions
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / Parameter("infectious_period"),
        source="Susceptible", 
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("latent_period"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("infectious_period"), 
        source="Infectious", 
        dest="Immune",
    )
    model.add_crude_birth_flow(
        "births",
        Parameter("crude_birth_rate") / 365.,
        "Susceptible",
    )
    model.add_universal_death_flows(
        "universal_death",
        death_rate=1. / Parameter("life_expectancy") / 365.,
    )
    
    # Outputs
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression",
    )
    model.request_output_for_compartments(
        name="total_population",
        compartments=compartments,
    )
    model.request_function_output(
        name="incidence_rate",
        func=DerivedOutput("incidence") / DerivedOutput("total_population") * 1e5,
    )
        
    return model

In [ ]:
seir_variable_births_model = build_demog_model(model_config)

#### Panel a)

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
line_styles = ("-", "--", ":")
for r, rate in enumerate((0.015, 0.025, 0.04)):
    parameters.update({"crude_birth_rate": rate})
    seir_variable_births_model.run(parameters=parameters)
    incidence_rate = seir_variable_births_model.get_derived_outputs_df()["incidence_rate"]
    left_ax.plot(
        incidence_rate.index / 365.,
        incidence_rate,
        color="k",
        linestyle=line_styles[r],
        label=int(rate * 1000),
    )
left_ax.legend(title="Annual birth rate per 1000")
left_ax.set_xlabel("Time (years)")
left_ax.set_xlim(20., 70.)
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_ylabel("Number of infectious persons/100,000/day")
left_ax.set_ylim(0., 15.)
left_ax.set_yticks(range(0, 20, 5))
left_ax.spines.top.set_visible(False)

image = plt.imread("./figures/fig_4_26.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");